In [7]:
from get_radiance_sum import *

## NTL using raw data from NASA

In [8]:
year = 2013
month=None
ntl_filename = "VNL_v2_npp_2013_global_vcmcfg_c202101211500.average.tif"

In [10]:
ntl_filepath = "./data/earth_observation_group/"

if month is not None:
    ntl_filepath += "monthly/"+str(year)+"/"+str(month)+"/"
else:
    ntl_filepath += "annual/"+str(year)+"/"
    
    
ntl_filepath += ntl_filename

region_shapefilepath = "./data/country_geometry/all_country/ne_10m_admin_0_countries.shp"

In [12]:
%timeit -n 1 ntl_df = run(year, month, ntl_filepath, region_shapefilepath, n_procs=6)

Process: 1:  21%|██████████▊                                          | 8/39 [07:49<30:20, 58.73s/it]
Process Process-38:
Traceback (most recent call last):
  File "/home/ankur/miniconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/ankur/miniconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ankur/Ankur/CLS/thesis/get_radiance_sum.py", line 27, in get_polygon_radiance_value
    data = np.reshape(out_image.data.tolist(), (out_image.shape[1], out_image.shape[2]))
  File "/home/ankur/Ankur/CLS/thesis/get_radiance_sum.py", line 37, in get_polygon_radiance_value_for_mproc
    res = get_polygon_radiance_value(rasterio_img, polygon)
  File "<__array_function__ internals>", line 180, in reshape
  File "/home/ankur/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py", line 298, in reshape
    return _wrapfunc(a, 'reshape', newshape, order=order)
  File "/home/

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/ankur/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_28784/3398113772.py", line 1, in <cell line: 1>
    get_ipython().run_line_magic('timeit', '-n 1 ntl_df = run(year, month, ntl_filepath, region_shapefilepath, n_procs=6)')
  File "/home/ankur/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2305, in run_line_magic
    result = fn(*args, **kwargs)
  File "/home/ankur/.local/lib/python3.9/site-packages/IPython/core/magics/execution.py", line 1166, in timeit
    all_runs = timer.repeat(repeat, number)
  File "/home/ankur/miniconda3/lib/python3.9/timeit.py", line 205, in repeat
    t = self.timeit(number)
  File "/home/ankur/.local/lib/python3.9/site-packages/IPython/core/magics/execution.py", line 156, in timeit
    timing = self.inner(it, self.timer)
  File "<magic-timeit>", line 1, in inner

## Postprocessing of NTL data

### 1. Dropping the countries for which GDP data is not available

In [ ]:
ntl_df.to_csv("./data/earth_observation_group/annual/2013/countries_ntl_sum.csv", index=False)

### 2. Setting negative values of NTL and low values(using std dev) to zero. Will be dealt with imputation later.

## NTL using Google Earth Engine API

In [1]:
import google
import geemap, ee
from geemap import foliumap

In [2]:
# ee.Authenticate()
credentials, project_id = google.auth.default()

/home/ankur/miniconda3/lib/python3.9/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
ee.Initialize(credentials, project="thesis-ntl-ankursatya")

# Check for the working of Google EE API
print(ee.Image("NASA/NASADEM_HGT/001").get("title").getInfo())

NASADEM: NASA NASADEM Digital Elevation 30m


In [7]:
turk_geom = ee.Feature(ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Turkey')).first()).geometry()

In [8]:
def get_region_geometry(country_name, admin_level):
    region_geometry = ee.Feature(ee.FeatureCollection("FAO/GAUL/2015/level{}".format(admin_level)).filter(
        ee.Filter.eq('ADM{}_NAME'.format(admin_level), country_name)).first()).geometry()
    return region_geometry

def get_ntl_mean_region(region_name, region_admin_level, start_date, end_date):
    """
        Parameters
        ----------
        region_name: str
            Official name of the region like Turkey.
        region_admin_level: int
            Administritative level of the region. For example a country would be level 0, province level 1.
        start_date : str
            starting date in the format 'YYYY-MM-DD'.
        end_date : str
             ending date in the format 'YYYY-MM-DD'.
             
        Returns
        ---------
        NTL value for all the pixels in the region aggregated over the year.
    """
    
    # define ImageCollection id
    viirs_rc_id = "NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG"

    # create an ee object for our image
    viirs_collection = ee.ImageCollection(viirs_rc_id).filterDate(start_date, end_date)
    # view ImageCollection contents
    viirs_collection.getInfo()
    
    # Sums the value for all the months. Each image in the collection is the aggregation of NTL of daily
    # values for a month. So summing over all the 12 months will give us the aggregation for the
    # complete year.
    viirs_img = viirs_collection.reduce(ee.Reducer.sum())
    visrange = {
      min: 0,
      max: 60
    }
    
    # Getting the region geometry
    region_geometry = get_region_geometry(region_name, region_admin_level)
    
    # Reduce the region. The region parameter is the Feature geometry.
    ntl_mean = viirs_img.reduceRegion(reducer=ee.Reducer.sum(), geometry=region_geometry, scale=30, 
                                      maxPixels=1e12)

    return ntl_mean
    

In [9]:
region_name = "Turkey"
region_admin_level = 0
start_date = '2013-01-01'
end_date = '2013-03-01'

ntl_mean = get_ntl_mean_region(region_name, region_admin_level, start_date, end_date)

In [17]:
ntl_mean.getInfo()["avg_rad_sum"]

1953541081.953666

In [10]:
ntl_mean

In [8]:
center_lon = 28.95
center_lat = 41.03
zoom_level = 9

tmp_map = foliumap.Map(center=[center_lat,center_lon], zoom=zoom_level)

In [9]:
# tmp_map.addLayer(turk_geom, {}, "Turkey Administrative Area")
# tmp_map

In [29]:
# define ImageCollection id
viirs_rc_id = "NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG"

# create an ee object for our 2012 image
start_date = '2012-04-01'
end_date = '2012-12-31'

viirs12_collection = ee.ImageCollection(viirs_rc_id).filterDate(start_date, end_date)
# view ImageCollection contents
# viirs12_collection.getInfo()

In [30]:
viirs12_img = viirs12_collection.reduce(ee.Reducer.sum())
visrange = {
  min: 0,
  max: 60
}

In [33]:
tmp_map.addLayer(viirs12_img, vis_params=visrange, name='VIIRS Radiance Calibrated 2012')

tmp_map.addLayerControl()
# tmp_map

In [34]:
# map the clipped radiance calibrated nighttime lights
map1 = foliumap.Map()
map1.centerObject(turk_geom, zoom=zoom_level)

# use another basemap for additional options
map1.add_basemap('SATELLITE')
map1.addLayer(viirs12_img, {}, "Radiance Calibrated 2012")
map1.addLayerControl()
# map1

In [35]:
## Reduce the region. The region parameter is the Feature geometry.
ntl_mean = viirs12_img.reduceRegion(reducer=ee.Reducer.sum(), 
                                    geometry=turk_geom, scale=30, maxPixels=1e12);


In [36]:
ntl_mean

In [145]:
ntl_mean.values()# order: avg_rad_mean, cf_cvg_mean